In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
from tensorflow.keras.applications import VGG16
from tensorflow.keras.layers import Flatten, Dense, Dropout
from tensorflow.keras.models import Model

# Load VGG16 as the base model
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Freeze the base model layers
base_model.trainable = False

# Adding custom layers on top of the base model
x = base_model.output
x = Flatten()(x)  # Flatten the output layer to 1 dimension
x = Dense(512, activation='relu')(x)
x = Dropout(0.5)(x)  # Add dropout to prevent overfitting
predictions = Dense(10, activation='softmax')(x)

# Combine the base model and the custom layers
model = Model(inputs=base_model.input, outputs=predictions)

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()


58889256/58889256 [==============================] - 0s 0us/step
Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                              

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import os
import pandas as pd

# Load the driver image list
df = pd.read_csv('/content/drive/My Drive/Acads/MEngAI/CS 239/Datasets/Distracted Driving/driver_imgs_list.csv')

df['img'] = df.apply(lambda row: os.path.join(row['classname'], row['img']), axis=1)

# Verify the DataFrame
print(df.head())

# Check if the paths exist
for img_path in df['img']:
    if not os.path.exists(os.path.join('/content/drive/My Drive/Acads/MEngAI/CS 239/Datasets/Distracted Driving/imgs/train', img_path)):
        print(f"File not found: {img_path}")

# Setup the ImageDataGenerator for data augmentation
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=10,
    width_shift_range=0.1,
    height_shift_range=0.1,
    shear_range=0.1,
    zoom_range=0.2,
    horizontal_flip=False,
    fill_mode='nearest',
    validation_split=0.2  # Using 20% of data for validation
)

# Training generator
train_generator = train_datagen.flow_from_dataframe(
    dataframe=df,
    directory='/content/drive/My Drive/Acads/MEngAI/CS 239/Datasets/Distracted Driving/imgs/train',
    x_col='img',
    y_col='classname',
    subset='training',
    batch_size=32,
    seed=42,
    shuffle=True,
    class_mode='categorical',
    target_size=(224, 224)
)

# Validation generator
validation_generator = train_datagen.flow_from_dataframe(
    dataframe=df,
    directory='/content/drive/My Drive/Acads/MEngAI/CS 239/Datasets/Distracted Driving/imgs/train',
    x_col='img',
    y_col='classname',
    subset='validation',
    batch_size=32,
    seed=42,
    shuffle=True,
    class_mode='categorical',
    target_size=(224, 224)
)

  subject classname               img
0    p002        c0  c0/img_44733.jpg
1    p002        c0  c0/img_72999.jpg
2    p002        c0  c0/img_25094.jpg
3    p002        c0  c0/img_69092.jpg
4    p002        c0  c0/img_92629.jpg
Found 17940 validated image filenames belonging to 10 classes.
Found 4484 validated image filenames belonging to 10 classes.


In [5]:
history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // train_generator.batch_size,
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // validation_generator.batch_size,
    epochs=10
)

Epoch 1/10
266/560 [=============>................] - ETA: 1:03:14 - loss: 2.0964 - accuracy: 0.3517

KeyboardInterrupt: 

In [ ]:
from tensorflow.keras.applications import MobileNet
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense, Dropout
from tensorflow.keras.models import Model

# Load MobileNet as the base model
base_model = MobileNet(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Freeze the base model
base_model.trainable = False

# Add custom layers
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
x = Dropout(0.5)(x)
predictions = Dense(10, activation='softmax')(x)

# This is the model we will train
model = Model(inputs=base_model.input, outputs=predictions)

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()


In [ ]:
from tensorflow.keras.applications import VGG16, ResNet50, MobileNet

# Load pre-trained model without the top layer (head)
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Freeze the layers
for layer in base_model.layers:
    layer.trainable = False

# Add your head for fine-tuning
from tensorflow.keras import layers, models

model = models.Sequential([
    base_model,
    layers.GlobalAveragePooling2D(),
    layers.Dense(256, activation='relu'),
    layers.Dropout(0.5),
    layers.Dense(1, activation='sigmoid')  # Assuming binary classification
])

model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 vgg16 (Functional)          (None, 7, 7, 512)         14714688  
                                                                 
 global_average_pooling2d_1  (None, 512)               0         
  (GlobalAveragePooling2D)                                       
                                                                 
 dense_2 (Dense)             (None, 256)               131328    
                                                                 
 dropout_1 (Dropout)         (None, 256)               0         
                                                                 
 dense_3 (Dense)             (None, 1)                 257       
                                                                 
Total params: 14846273 (56.63 MB)
Trainable params: 131585 (514.00 KB)
Non-trainable params: 14714688 (56.13 MB)
_______

In [ ]:
# Unfreeze some of the top layers of the base model
base_model.trainable = True
fine_tune_at = 100  # Example: Unfreeze from this layer upwards

for layer in base_model.layers[:fine_tune_at]:
    layer.trainable = False

# Compile and train the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Assuming you have your training and validation data ready
history = model.fit(train_data, epochs=10, validation_data=val_data)

NameError: name 'train_data' is not defined

In [ ]:
from tensorflow.keras.regularizers import l2

# Add L2 regularization to a layer
layer = layers.Dense(256, activation='relu', kernel_regularizer=l2(0.01))